In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import json
import random
import matplotlib.pyplot as plt
from citipy import citipy
import requests
from config import weather_api_key

In [2]:
# Create a new set of 2,000 random latitudes and longitudes.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

718

In [5]:
import time
from datetime import datetime

In [6]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [7]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]

        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description})


# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | kapaa
Processing Record 2 of Set 1 | madhipura
Processing Record 3 of Set 1 | kungurtug
Processing Record 4 of Set 1 | souillac
Processing Record 5 of Set 1 | chokurdakh
Processing Record 6 of Set 1 | talara
Processing Record 7 of Set 1 | hermanus
Processing Record 8 of Set 1 | ponta do sol
Processing Record 9 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 10 of Set 1 | bluff
Processing Record 11 of Set 1 | honiara
Processing Record 12 of Set 1 | portland
Processing Record 13 of Set 1 | rikitea
Processing Record 14 of Set 1 | tsihombe
City not found. Skipping...
Processing Record 15 of Set 1 | tuban
Processing Record 16 of Set 1 | jumla
Processing Record 17 of Set 1 | kapra
Processing Record 18 of Set 1 | port elizabeth
Processing Record 19 of Set 1 | amderma
City not found. Skipping...
Processing Record 20 of Set 1 | aklavik
Processing Record 21 of Set 1 | bethel
Processing

In [8]:
# Convert to DataFrame
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Kapaa,US,22.0752,-159.3190,78.48,71,96,8.01,overcast clouds
1,Madhipura,IN,25.9167,86.7833,85.17,12,65,7.96,broken clouds
2,Kungurtug,RU,50.5994,97.5228,19.27,93,22,2.15,few clouds
3,Souillac,MU,-20.5167,57.5167,75.58,88,75,11.50,light intensity shower rain
4,Chokurdakh,RU,70.6333,147.9167,7.61,96,100,3.69,overcast clouds
5,Talara,PE,-4.5772,-81.2719,81.36,54,75,18.41,broken clouds
6,Hermanus,ZA,-34.4187,19.2345,52.81,64,81,15.39,broken clouds
7,Ponta Do Sol,PT,32.6667,-17.1000,66.70,74,100,3.20,overcast clouds
8,Bluff,NZ,-46.6000,168.3333,52.81,67,68,33.69,broken clouds
9,Honiara,SB,-9.4333,159.9500,77.34,88,100,1.52,overcast clouds


In [9]:
len(city_data)

656

In [11]:
# Create the output file (CSV)
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")